In [1]:
import httpx
import asyncio
import spacy
from openai import OpenAI
from dotenv import load_dotenv
import os
from nltk.stem import WordNetLemmatizer
import lemminflect
import time
import networkx as nx
from collections import defaultdict
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from nltk.corpus import wordnet as wn



load_dotenv()
#openAI_api_key = "ENTER YOUR API KEY HERE"
openAI_api_key = os.environ.get("OPENAI_API_KEY")
openAI_client = OpenAI(api_key=openAI_api_key)

# Download necessary data for WordNetLemmatizer if we haven't already
try:
    WordNetLemmatizer().lemmatize("test") # Just a test to trigger lookup error if not downloaded
except LookupError:
    nltk.download('wordnet')
    nltk.download('omw-1.4') # Open Multilingual Wordnet, often needed for full WordNet functionality

In [91]:
allowed_word_types = [
    "ADJ",	
    "ADV",
    "INTJ",	
    "NOUN",
    "PROPN",	
    "VERB"
]

NOUN_RELS = ["RelatedTo", "CapableOf", "IsA", "UsedFor", "AtLocation", "HasPrerequisite", "HasProperty", "ReceivesAction", "CreatedBy", "Causes", "HasA", "MadeOf"]
NOUN_REV_RELS = ["AtLocation", "IsA", "PartOf"]
VERB_RELS = ["MannerOf", "HasSubevent", "MotivatedByGoal", "IsA", "HasFirstSubevent", "HasLastSubevent"]
VERB_REV_RELS = ["CapableOf", "MannerOf", "CausesDesire", "UsedFor", "CreatedBy", "ReceivesAction", "HasFirstSubevent"]
ADJ_RELS = ["SimilarTo", "RelatedTo", "HasProperty"] 


stopwords = set([
    'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than', 'get', 'put',
])


In [137]:
lemmatizer = WordNetLemmatizer()
def get_word_lemma(word : str, pos_hint : str = None) -> str:
    """
    Gets the lemma of a single word without sentence context using NLTK's WordNetLemmatizer.
    pos_hint can be 'n' (noun), 'v' (verb), 'a' (adjective), 'r' (adverb).
    Defaults to 'n' if no hint is given.
    """
    return lemmatizer.lemmatize(word.lower(), pos=pos_hint) if pos_hint else lemmatizer.lemmatize(word.lower())

def get_all_possible_lemmas(word: str) -> list[str]:
    res = []
    for hint in ["v", "n", "a", "r", "s"]:
        res.append(get_word_lemma(word, hint))
    return res

def get_all_inflections(word : str) -> list[str]:
    inflections = lemminflect.getAllInflections(word)
    res = []
    for _, i in inflections.items():
        res += list(i)
    return res

def get_word_classes(word : str) -> set:
    pos_set = set()
    for synset in wn.synsets(word):
        #print(f"Synset: {synset}")
        if synset.name().split('.')[0] != word:
            continue  # Only consider exact matches
        if synset.pos() == 'n':
            pos_set.add('noun')
        elif synset.pos() == 'v':
            pos_set.add('verb')
        elif synset.pos() == 's':
            pos_set.add('adj')

    if len(pos_set) == 0: pos_set.add('noun') # default to noun if unidentified 
    return pos_set

def remove_stopwords(words : str) -> str:
    word_list = words.split()
    filtered_words = [word for word in word_list if word.lower() not in stopwords]
    return " ".join(filtered_words)
    

class Codemaster:
    def __init__(self, our_words : list[str], enemy_words : list[str], civilian_words : list[str], assassin_word : str, my_team : str):
        self.our_words = our_words
        self.enemy_words = enemy_words
        self.civilian_words = civilian_words
        self.assassin_word = assassin_word
        self.my_team = my_team
        self.all_board_words = set(our_words + enemy_words + civilian_words + [assassin_word])

        self.clue_candidates = defaultdict(list)
        self.min_clue_weight_threshold = 1.4

        # Spacy
        self.nlp = spacy.load("en_core_web_trf")

        # API clients
        self.conceptnet_client = httpx.AsyncClient(http2=True)
        self.openai_client = OpenAI(api_key=openAI_api_key)

        # All forms of board words
        self.lemmas = set()
        for word in self.all_board_words:
            self.lemmas.update(word)
            self.lemmas.update(get_all_inflections(word))
            self.lemmas.update(get_all_possible_lemmas(word))

        # Create graph from ConceptNet queries
        self.graph = nx.DiGraph()


    async def fetch_conceptnet(self, url : str) -> dict:
        r = await self.conceptnet_client.get(url, follow_redirects=True)
        return r.json()
    
    def _filter(self, words: str) -> list[str]:
        """
        return the filtered list of words. Each word must NOT appear as a lemma or an inflection of any words on the board (which is given by self.lemmas).
        """
        doc = self.nlp(words)
        l = list(filter(lambda token: (token.pos_ in allowed_word_types) and (token.lemma_ not in self.lemmas) and (token.text not in self.lemmas), doc)) 
        # l is a list of tokens, convert them to string to return
        return [token.text for token in l]
    
    def _process_edge(self, edge : dict, target_word):
        """
        for a given target word, and a given edge it has on ConceptNet, process the node this edge points to:
            + First, the node has to be in English.
            + Second, the node may be multi-word, so we need to process each individual word in it. 
            Keep the open class words (adjective, nouns, verbs, etc) as specified in the universal POS tags: https://universaldependencies.org/u/pos/
            + Then, we need to make sure that each word follows the rules of the game (no subword embedding, etc).
        The `self.filter` function performs the last 2 filters.
        """

        # find whether our target word is at the start node or end node of this edge
        start_node = edge["start"]
        end_node = edge["end"]

        start_node_label = remove_stopwords(start_node["label"])
        end_node_label = remove_stopwords(end_node["label"])
        
        if target_word in start_node_label:
            if end_node["language"] != "en": return []
            label_words = self._filter(end_node_label)
        else:
            if start_node["language"] != "en": return []
            label_words = self._filter(start_node_label)
        return label_words

    async def _fetch_relations(self, target_word: str, rel_list: list, is_rev: bool = False) -> set:
        """Generic function to fetch and process edges for a list of relations. Create multiple parallel tasks to process these relations."""
        clues = set()
        tasks = []
        for rel in rel_list:
            node_param = "end" if is_rev else "start"
            url = f"http://api.conceptnet.io/query?{node_param}=/c/en/{target_word}&rel=/r/{rel}&limit=10"
            tasks.append(self.conceptnet_client.get(url, follow_redirects=True))

        responses = await asyncio.gather(*tasks, return_exceptions=True)
        
        for i, res in enumerate(responses):
            if isinstance(res, Exception):
                # print(f"Warning: Request for {target_word} with relation {rel_list[i]} failed: {res}")
                continue
            
            try:
                api_res = res.json()
                relation_name = rel_list[i]
                for edge in api_res.get("edges", []):
                    processed_clues = self._process_edge(edge, target_word)
                    for clue in processed_clues:
                        clues.add((clue, relation_name, is_rev))
            except (ValueError, KeyError) as e:
                # print(f"Warning: Could not parse JSON for {target_word} with relation {rel_list[i]}: {e}")
                continue
        return clues

    
    async def _fetch_clues_for_word(self, word: str, specific_rels: list = None) -> dict:
        """
        Fetches all potential clues for a single word. Depending on its POS (Part Of Speech), use the relation list accordingly.
        If specific_rels is provided, use those relations instead.
        """
        clues_by_pos = {'noun': set(), 'verb': set(), 'adj': set()}
        word_classes = get_word_classes(word)

        for pos in word_classes:
            rels, rev_rels = [], []
            if specific_rels:
                rels = specific_rels
                rev_rels = specific_rels
            elif pos == 'noun':
                rels, rev_rels = NOUN_RELS, NOUN_REV_RELS
            elif pos == 'verb':
                rels, rev_rels = VERB_RELS, VERB_REV_RELS
            elif pos == 'adj':
                rels, rev_rels = ADJ_RELS, []

            forward_clues, reverse_clues = await asyncio.gather(
                self._fetch_relations(word, rels, is_rev=False),
                self._fetch_relations(word, rev_rels, is_rev=True)
            )
            clues_by_pos[pos].update(forward_clues)
            clues_by_pos[pos].update(reverse_clues)
        return clues_by_pos

    async def build_clue_graph(self):
        """Builds the clue graph by querying ConceptNet up to 2 levels deep."""     

        total_start = time.time()
        
        # Add all board words first
        for word in self.our_words: self.graph.add_node(word, type='our_word')
        for word in self.enemy_words: self.graph.add_node(word, type='enemy_word')
        for word in self.civilian_words: self.graph.add_node(word, type='civilian_word')
        self.graph.add_node(self.assassin_word, type='assassin_word')

        # Use a temporary dict to store max weights before sorting
        temp_candidates = defaultdict(dict)

        # Step 1: populate graph with depth-1 clues
        print("\n--- Building Clue Graph (Depth 1) ---")
        d1_tasks = []
        for word in self.our_words:
            d1_tasks.append(self._fetch_clues_for_word(word))
        
        d1_results = await asyncio.gather(*d1_tasks)
        
        d1_clues_for_stage2 = []
        for i, clues_by_pos in enumerate(d1_results):
            source_word = self.our_words[i]
            # Check if the word is a landmark
            doc = self.nlp(source_word)
            is_landmark = doc.ents and doc.ents[0].label_ in ["GPE", "LOC"]
            
            for pos, clue_tuples in clues_by_pos.items():
                for clue, relation, is_rev in clue_tuples:
                    if clue in self.all_board_words: continue

                    weight = self.relation_weights.get(pos, {}).get(relation, 1.3)
                    if is_rev: weight *= self.reverse_relation_penalty

                    
                    if is_landmark and relation == 'AtLocation':
                        weight *= self.landmark_location_bonus
                    
                    if weight > self.min_clue_weight_threshold:
                        # Add to graph, replacing edge if new one is stronger
                        if not self.graph.has_edge(source_word, clue):
                            self.graph.add_edge(source_word, clue, weight=weight)
                        else:
                            if weight > self.graph[source_word][clue]['weight']:
                                self.graph.remove_edge(source_word, clue)
                                self.graph.add_edge(source_word, clue, weight=weight)

                        current_max = temp_candidates[source_word].get(clue, -1)
                        temp_candidates[source_word][clue] = max(current_max, weight)
                        d1_clues_for_stage2.append((source_word, clue, pos, relation))

        print(f"Graph after Depth 1: {self.graph.number_of_nodes()} nodes, {self.graph.number_of_edges()} edges.")
        print(f"Time taken: {time.time() - total_start}")

        print("\n--- Building Clue Graph (Depth 2) ---")
        start_2 = time.time()
        d2_tasks = []
        for source_word, d1_clue, pos, rel in d1_clues_for_stage2:
            d2_tasks.append(self._fetch_clues_for_word(d1_clue, specific_rels=[rel]))
        
        d2_results = await asyncio.gather(*d2_tasks)

        for i, clues_by_pos in enumerate(d2_results):
            source_word, d1_clue, pos, _ = d1_clues_for_stage2[i]
            d1_weight = self.graph[source_word][d1_clue]['weight']

            for _, clue_tuples in clues_by_pos.items():
                for d2_clue, d2_relation, is_rev in clue_tuples:
                    if d2_clue in self.all_board_words: continue

                    d2_weight = self.relation_weights.get(pos, {}).get(d2_relation, 1.3)
                    if is_rev: d2_weight *= self.reverse_relation_penalty
                    
                    total_path_weight = (d1_weight + (d2_weight * 0.8)) / 2 # Apply decay for depth 2

                    if total_path_weight > self.min_clue_weight_threshold:
                        # Add to graph, replacing edge if new one is stronger
                        if not self.graph.has_edge(d1_clue, d2_clue):
                            self.graph.add_edge(d1_clue, d2_clue, weight=d2_weight)
                        else:
                            if d2_weight > self.graph[d1_clue][d2_clue]['weight']:
                                self.graph.remove_edge(d1_clue, d2_clue)
                                self.graph.add_edge(d1_clue, d2_clue, weight=d2_weight)
                        
                        current_max = temp_candidates[source_word].get(d2_clue, -1)
                        temp_candidates[source_word][d2_clue] = max(current_max, total_path_weight)
        print(f"Finished stage 2.\nTime taken: {time.time() - start_2}")

        # --- Finalize: Sort and store the candidate lists ---
        for word, candidates in temp_candidates.items():
            sorted_list = sorted(candidates.items(), key=lambda item: item[1], reverse=True)
            self.clue_candidates[word] = [(weight, clue) for clue, weight in sorted_list]
        
        print("\n--- Finished building graph ---")
        print(f"Total time taken: {time.time() - total_start}")

        
    def find_candidate_clues(self, target_words: list[str]) -> set:
        """
        Finds candidate clues by intersecting the pre-computed clue lists.
        """
        if not target_words:
            return set()

        # Get the list of set of candidate clues for each target word
        try:
            candidate_sets = [set(clue for _, clue in self.clue_candidates[word]) for word in target_words]
        except KeyError:
            return set() # A target word might have no valid clues
        
        if not candidate_sets:
            return set()

        # Get intersection of all the sets
        intersection = candidate_sets[0].intersection(*candidate_sets[1:])
        return intersection

    def calculate_graph_score(self, clue: str, target_words: list[str]) -> float:
        """
        Calculates a score based on the pre-computed weights in the candidate lists.
        """
        total_weight = 0.0
        
        # Create a quick lookup map for each target's clues
        candidate_maps = {word: {c: w for w, c in self.clue_candidates.get(word,)} for word in target_words}

        for word in target_words:
            # Add the pre-computed max path weight for this clue
            total_weight += candidate_maps[word].get(clue, 0)

        if total_weight == 0:
            return 0.0
        
        # Reward clues that connect more words
        return total_weight * (len(target_words) ** 2)


In [138]:
start = time.time()
blue_words = ["dwarf", "foot", "moon", "star", "ghost", "beijing", "fighter", "roulette", "alps"]
#red_words = ["club", "superhero", "mount", "bomb", "knife", "belt", "robot", "rock", "bar", "lab"]
red_words = ["drive"]
civilian_words = ["dead"]
assassin_word = "agent"
master = Codemaster(blue_words, red_words, civilian_words, assassin_word, "red")
candidates = await master.build_clue_graph()

    
print(f"Time taken: {time.time() - start}")


--- Building Clue Graph (Depth 1) ---
Graph after Depth 1: 285 nodes, 292 edges.
Time taken: 9.568947792053223

--- Building Clue Graph (Depth 2) ---
Finished stage 2.
Time taken: 111.04610061645508

--- Finished building graph ---
Total time taken: 120.61553025245667
Time taken: 122.03094983100891


In [139]:
for word, clues in master.clue_candidates.items():
    print(f"------ {word} ---------")
    for clue in clues:
        print(f"Clue: {clue[1]}, Weight: {clue[0]}") 

------ dwarf ---------
Clue: organism, Weight: 2.0
Clue: person, Weight: 2.0
Clue: living, Weight: 2.0
Clue: mythical, Weight: 2.0
Clue: biped, Weight: 2.0
Clue: animal, Weight: 2.0
Clue: short, Weight: 2.0
Clue: small, Weight: 2.0
Clue: human, Weight: 2.0
Clue: sentient, Weight: 2.0
Clue: stunt, Weight: 2.0
Clue: creature, Weight: 1.8
Clue: race, Weight: 1.8
Clue: thing, Weight: 1.8
Clue: system, Weight: 1.8
Clue: homo, Weight: 1.8
Clue: category, Weight: 1.8
Clue: causal, Weight: 1.8
Clue: complex, Weight: 1.8
Clue: member, Weight: 1.8
Clue: legal, Weight: 1.8
Clue: individual, Weight: 1.8
Clue: sapien, Weight: 1.8
Clue: body, Weight: 1.8
Clue: social, Weight: 1.8
Clue: grammatical, Weight: 1.8
Clue: Living, Weight: 1.8
Clue: people, Weight: 1.8
Clue: legged, Weight: 1.8
Clue: genus, Weight: 1.8
Clue: diplont, Weight: 1.8
Clue: heterotroph, Weight: 1.8
Clue: object, Weight: 1.8
Clue: non, Weight: 1.8
Clue: preditor, Weight: 1.8
Clue: perceptual, Weight: 1.8
Clue: artifactual, Weight:

In [181]:
l = ['drive', 'dwarf', 'foot', 'moon', 'star', 'ghost', 'beijing', 'fighter', 'roulette', 'alp', 'dead', 'agent']
nlp = spacy.load("en_core_web_trf")
doc = nlp("driving")
for token in doc:
    if token.lemma_ not in l:
        print(token.text, token.lemma_)

driving driving


In [164]:
response = openai_client.responses.create(
    model="gpt-4.1",
    input = f"""The following are possible word classes: [ADV (adverb), ADJ (adjective), INTJ (interjection), NOUN (noun), PROPN (proper noun), VERB (verb)].
    
    Determine which of these classes the word 'table' belongs to based on its possible usages in English (and English only).
    
    At the end of your response, list the corresponding abbreviations (e.g., NOUN, VERB), separated by commas, on a single line.
    """

).output_text.split("\n")[-1]

In [165]:
response

'NOUN, VERB'

In [125]:

word = "VOLDEMORT"
response = openAI_client.responses.create(
    model="gpt-4.1-nano",
    input = f"""
The possible word classes are: [ADJ (adjective), NOUN (noun), VERB (verb)].

Given the word {word}, identify which of these classes it can belong to based solely on its usage in English and English only.

Be as quick as you can. Respond with only the applicable abbreviations (e.g., NOUN, VERB), separated by commas, and nothing else.
"""

).output_text.split("\n")[-1]

print(response)

NOUN


In [107]:
from nltk.corpus import wordnet as wn

def get_pos(word):
    pos_set = set()
    for synset in wn.synsets(word):
        print(f"Synset: {synset}")
        if synset.name().split('.')[0] != word:
            continue  # Only consider exact matches
        if synset.pos() == 'n':
            pos_set.add('noun')
        elif synset.pos() == 'v':
            pos_set.add('verb')
        elif synset.pos() == 's':
            pos_set.add('adj')

    if len(pos_set) == 0:
        
            
    return pos_set

# Example usage
word = "DESTINY"
pos_tags = get_pos(word)
print(f"{word} can be used as: {', '.join(pos_tags)}")

Synset: Synset('destiny.n.01')
Synset: Synset('destiny.n.02')
Synset: Synset('fortune.n.04')
DESTINY can be used as: 


In [105]:
remove_stopwords("a city in France")

'city France'

9
Time taken: 8.19223427772522
